In [15]:
import os

from dotenv import load_dotenv

load_dotenv('keys.env')

True

In [16]:
companies = ['KORCSMAROSNE SZILAGYI ELZA',
'BRANDFOLDER, INC.',
'Carine Pascal',
'Duales System Deutschland GmbH',
'Elizabeth Epp',
'Farene Ng',
'Ihap Ahmed',
'Ing. Petr Herník',
'NGAME SERVICES LIMITED',
'Xiaojing Li']

In [17]:
import importlib

import models.GeminiClient
import src.KPIExtractor

importlib.reload(src.KPIExtractor)
importlib.reload(models.GeminiClient)

from models.GeminiClient import GeminiClient
from src.KPIExtractor import KPIExtractor

llmClient = GeminiClient(os.getenv('GEMINI_API_KEY'))
kpi_extractor = KPIExtractor(llmClient)
result_df = kpi_extractor.extract_kpis(companies)

Extract the Financial KPIs for these companies below:
Companies:
- KORCSMAROSNE SZILAGYI ELZA
- BRANDFOLDER, INC.
- Carine Pascal
- Duales System Deutschland GmbH
- Elizabeth Epp
- Farene Ng
- Ihap Ahmed
- Ing. Petr Herník
- NGAME SERVICES LIMITED
- Xiaojing Li

KPIs:        
- Total Revenue
- Revenue Growth Rate
- Recurring Revenue (ARR/MRR)
- Revenue per Customer
- Gross Profit
- Gross Margin
- Net Profit
- Net Profit Margin
- EBITDA
- Operating Profit Margin
- Cost of Goods Sold (COGS)
- Operating Expenses (OPEX)
- Cost-to-Revenue Ratio
- Free Cash Flow (FCF)
- Operating Cash Flow
- Cash Conversion Cycle (CCC)
- Total Assets
- Total Liabilities
- Equity
- Debt-to-Equity Ratio
- Current Ratio
- Market Capitalization
- Enterprise Value (EV)
- Price-to-Earnings (P/E) Ratio
- Return on Assets (ROA)
- Return on Equity (ROE)
- Asset Turnover Ratio
- Customer Acquisition Cost (CAC)
- Lifetime Value (LTV)
- LTV-to-CAC Ratio

In [18]:
result_df.to_csv('data/temp.csv', index=False)

In [19]:
import pprint
from openai import OpenAI

gemini_client = OpenAI(api_key=os.getenv("GEMINI_API_KEY"),
                       base_url="https://generativelanguage.googleapis.com/v1beta/")

chat_completion = gemini_client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=[
                {"role": "system", "content": """Help me find the stock tickers of the companies. If a company is subsidiary of another company, 
                 use the parent company's ticker if the subsidiary is not listed, like Brandfolder is subsidiary of Smartsheet, you can return Smartsheet's ticker with 
                    Brandfolder's name. These stock tickers will be used to query
                 finance data from yfinance so if a company is listed in some other country than US, make sure you suffix its ticker with .country so that
                 it can be used with yfinance('ticker') to fetch its finance KPIs. Return only the companies which are listed and output should be json
                 like {'company_name': 'ticker', ...}"""},  # Set the context
                {"role": "user", "content": f"find the stock tickers of following companies: {companies}"}   # User's prompt
    ])

pprint.pprint(chat_completion)
pprint.pprint(chat_completion.choices[0].message.content)

ChatCompletion(id=None, choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "BRANDFOLDER, INC.": "SMAR"\n}\n```\n', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738055525, model='gemini-1.5-flash', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=21, prompt_tokens=223, total_tokens=244, completion_tokens_details=None, prompt_tokens_details=None))
'```json\n{\n  "BRANDFOLDER, INC.": "SMAR"\n}\n```\n'


In [20]:
print((chat_completion.choices[0].message.content))

```json
{
  "BRANDFOLDER, INC.": "SMAR"
}
```



#### Extracting Metrics using yFinance.

In [21]:
import yfinance as yf

# Fetch company data using its ticker symbol
def get_kpi(ticker_symbol):
    company = yf.Ticker(ticker_symbol)

    kpi_list = ['totalRevenue', 'grossProfits', 'netIncomeToCommon', 
                'ebitda', 'totalAssets', 'totalLiabilities', 'marketCap']

    # Fetch KPIs for the company
    company_info = company.info

    for kpi in kpi_list:
        print(f"{kpi}: {company_info.get(kpi, None)}")


In [22]:
get_kpi('SMAR')

totalRevenue: 1042262976
grossProfits: 850561024
netIncomeToCommon: -42404000
ebitda: -53498000
totalAssets: None
totalLiabilities: None
marketCap: 7905913344


In [23]:
company = yf.Ticker('SMAR')
info = company.info
info

{'address1': '500 108th Ave NE',
 'address2': 'Suite 200',
 'city': 'Bellevue',
 'state': 'WA',
 'zip': '98004',
 'country': 'United States',
 'phone': '844 324 2360',
 'website': 'https://www.smartsheet.com',
 'industry': 'Software - Application',
 'industryKey': 'software-application',
 'industryDisp': 'Software - Application',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Smartsheet Inc. provides enterprise platform to plan, capture, manage, automate, and report on work for teams and organizations. The company offers Smartsheet, which includes Dashboards, Cardview, Grid, Reports, Projects, Calendar, Forms, Automations, and Integrations; WorkApps enables customers to build user-friendly apps; Smartsheet Advance; Smartsheet Safeguard; Connectors provides embedded integrations with industry-leading systems of record; Control Center enables organizations to achieve consistent work execution; and Dynamic View enables business u

In [24]:
company.balance_sheet.iloc[:, 0]


Ordinary Shares Number                                     136884011.0
Share Issued                                               136884011.0
Total Debt                                                  49811000.0
Tangible Book Value                                        436418000.0
Invested Capital                                           605855000.0
Working Capital                                            236019000.0
Net Tangible Assets                                        436418000.0
Capital Lease Obligations                                   49811000.0
Common Stock Equity                                        605855000.0
Total Capitalization                                       605855000.0
Total Equity Gross Minority Interest                       605855000.0
Stockholders Equity                                        605855000.0
Gains Losses Not Affecting Retained Earnings                 -146000.0
Other Equity Adjustments                                     -146000.0
Retain

In [25]:
import numpy as np


metrics = {
    "Gross Margin": 0.4,
    "Net Profit Margin": 0.3,
    "Operating Profit Margin": 0.5,
    "Current Ratio": 0.8,
    "Free Cash Flow": 0.7,
    "Debt-to-Equity Ratio": 0.4,
    "ROA": 0.6,
    "ROE": 0.7,
    "Revenue Growth Rate": 0.9,
    "Asset Turnover Ratio": 0.5
}

weights = {
    "Gross Margin": 0.1,
    "Net Profit Margin": 0.1,
    "Operating Profit Margin": 0.1,
    "Current Ratio": 0.1,
    "Free Cash Flow": 0.1,
    "Debt-to-Equity Ratio": 0.1,
    "ROA": 0.1,
    "ROE": 0.05,
    "Revenue Growth Rate": 0.15,
    "Asset Turnover Ratio": 0.1
}

# Calculate the weighted score
final_score = 5 * sum(metrics[metric] * weights[metric] for metric in metrics)
print(f"Final Score: {final_score:.2f}/5")

Final Score: 2.95/5
